In [ ]:
%matplotlib notebook
import pickle
import numpy as np
import math
import sk_geo_tools as sk
import matplotlib.pyplot as plt
import pg_fitter_tools as fit

In [ ]:
# Load fit results
with open('../../PhotogrammetryAnalysis_msekatchev/results/full_ring/SK_ring.pkl', 'rb') as input:
#with open('results/SK_demo3.pkl', 'rb') as input:
    fitter_all = pickle.load(input)

reco_raw = {f: fitter_all.reco_locations[i] for f, i in fitter_all.feature_index.items()}
expected_locations = {f: fitter_all.seed_feature_locations[i] for f, i in fitter_all.feature_index.items()}

# Kabsch transform
errors, reco_transformed, scale, R, translation, location_mean = fit.kabsch_errors(expected_locations, reco_raw)
reco_locations = {f: reco_transformed[i] for f, i in fitter_all.feature_index.items()}

In [ ]:
pmt_shift_distance_array = []
pmt_shift_distance_array_x = []
pmt_shift_distance_array_y = []
pmt_shift_distance_array_z = []
pmt_shift_distance_array_radial = []
pmt_shift_distance_array_tangential = []

labelled = False
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')

for chosenPMT in reco_locations.keys():
    if (chosenPMT[-2:] != "00"):
        continue
        
    #print("PMT ID:", chosenPMT)
    
    expected_3D_dynode_centre = expected_locations[chosenPMT]
    reconstructed_3D_dynode_centre = reco_locations[chosenPMT]

    #print("Rec. Dynode:", reconstructed_3D_dynode_centre)

    pmt_shift_distance = math.sqrt((reconstructed_3D_dynode_centre[0]-expected_3D_dynode_centre[0])**2 +(reconstructed_3D_dynode_centre[1]-expected_3D_dynode_centre[1])**2 +(reconstructed_3D_dynode_centre[2]-expected_3D_dynode_centre[2])**2  )
    pmt_shift_distance_array.append(pmt_shift_distance)
    pmt_shift_distance_array_x.append(reconstructed_3D_dynode_centre[0]-expected_3D_dynode_centre[0])
    pmt_shift_distance_array_y.append(reconstructed_3D_dynode_centre[1]-expected_3D_dynode_centre[1])
    pmt_shift_distance_array_z.append(reconstructed_3D_dynode_centre[2]-expected_3D_dynode_centre[2])

    vector = reconstructed_3D_dynode_centre - expected_3D_dynode_centre

    radius = math.sqrt(expected_3D_dynode_centre[0]**2 + expected_3D_dynode_centre[1]**2)

    radial_vector = (expected_3D_dynode_centre[0]/radius, expected_3D_dynode_centre[1]/radius, 0)
    pmt_shift_distance_array_radial.append(np.dot(vector, radial_vector))

    tangential_vector = (expected_3D_dynode_centre[1]/radius, -expected_3D_dynode_centre[0]/radius, 0)
    pmt_shift_distance_array_tangential.append(np.dot(vector, tangential_vector))
  
    #print("PMT shift:", pmt_shift_distance)
    #print("\n")
    
    if labelled == False:
        ax.scatter(reconstructed_3D_dynode_centre[0],reconstructed_3D_dynode_centre[1],reconstructed_3D_dynode_centre[2], color="red", marker = '*', label = "reconstructed dynode point")
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='red')

        ax.scatter(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], color="green", marker = '*', label = "expected dynode point")
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='green')
        labelled = True
    else:
        ax.scatter(reconstructed_3D_dynode_centre[0],reconstructed_3D_dynode_centre[1],reconstructed_3D_dynode_centre[2], color="red", marker = '*')
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='red')

        ax.scatter(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], color="green", marker = '*')
        X,Y,Z = [[expected_3D_dynode_centre[i], expected_3D_dynode_centre[i]] for i in range(3)]
        ax.plot(X,Y,Z,color='green')
    
    ax.text(expected_3D_dynode_centre[0],expected_3D_dynode_centre[1],expected_3D_dynode_centre[2], chosenPMT[:-3],size=8, zorder=4, color='k')
    #title = str(chosenPMT)+", distance = "+str(round(pmt_shift_distance,3))+" px"
    #print(str(chosenPMT)+", distance = "+str(round(pmt_shift_distance,3))+" px\n")
    #plt.title(title)
    
plt.legend(loc=0)
plt.show()

In [ ]:
nbins = round(len(pmt_shift_distance_array)/4)

In [ ]:
errorAverage = sum(pmt_shift_distance_array) / len(pmt_shift_distance_array)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array, nbins, density=False, facecolor='g')
plt.xlabel('Absolute Distance Error (cm)')
plt.ylabel('Probability')
printAverage = "Average = " + str(round(errorAverage,2))
plt.title(printAverage)

plt.grid(True)
plt.show()

In [ ]:
errorAverage_x = sum(pmt_shift_distance_array_x) / len(pmt_shift_distance_array_x)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array_x, nbins, facecolor='g')
plt.xlabel('x Distance Error (cm)', size=25)
plt.ylabel('Frequency', size=25)


STD_x = np.std(pmt_shift_distance_array_x)
STD_x_error = STD_x / np.sqrt(len(pmt_shift_distance_array_x))
printSTD = "SD = " + str(round(STD_x,1)) + " cm"
printAverage = "AVG = " + str(round(errorAverage_x,2)) + " ± " + str(round(STD_x_error,2)) + " cm"

plt.title(printAverage+", "+printSTD, size=30, wrap=True)

plt.tick_params(axis='both', labelsize=20)
plt.grid(True)
plt.show()
plt.savefig("error-x.png")

In [ ]:
errorAverage_y = sum(pmt_shift_distance_array_y) / len(pmt_shift_distance_array_y)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array_y, nbins, facecolor='g')
plt.xlabel('y Distance Error (cm)', size=25)
plt.ylabel('Frequency', size=25)

STD_y = np.std(pmt_shift_distance_array_y)
STD_y_error = STD_y / np.sqrt(len(pmt_shift_distance_array_y))
printSTD = "SD = " + str(round(STD_y,1)) + " cm"
printAverage = "AVG = " + str(round(errorAverage_y,2)) + " ± " + str(round(STD_y_error,2)) + " cm"

plt.title(printAverage+", "+printSTD, size=30, wrap=True)

plt.tick_params(axis='both', labelsize=20)
plt.grid(True)
plt.show()
plt.savefig("error-y.png")

In [ ]:
errorAverage_z = sum(pmt_shift_distance_array_z) / len(pmt_shift_distance_array_z)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array_z, nbins, facecolor='g')
plt.xlabel('z Distance Error (cm)', size=25)
plt.ylabel('Frequency', size=25)

STD_z = np.std(pmt_shift_distance_array_z)
STD_z_error = STD_z / np.sqrt(len(pmt_shift_distance_array_z))
printSTD = "SD = " + str(round(STD_z,1)) + " cm"
printAverage = "AVG = " + str(round(errorAverage_z,2)) + " ± " + str(round(STD_z_error,2)) + " cm"

plt.title(printAverage+", "+printSTD, size=30, wrap=True)

plt.tick_params(axis='both', labelsize=20)
plt.grid(True)
plt.show()
plt.savefig("error-z.png")

In [ ]:
errorAverage_radial = sum(pmt_shift_distance_array_radial) / len(pmt_shift_distance_array_radial)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array_radial, nbins, facecolor='g')
plt.xlabel('Radial Error (cm)', size=25)
plt.ylabel('Frequency', size=25)

STD_radial = np.std(pmt_shift_distance_array_radial)
STD_radial_error = STD_radial / np.sqrt(len(pmt_shift_distance_array_radial))
printSTD = "SD = " + str(round(STD_radial,1)) + " cm"
printAverage = "AVG = " + str(round(errorAverage_radial,2)) + " ± " + str(round(STD_radial_error,2)) + " cm"

plt.title(printAverage+", "+printSTD, size=30, wrap=True)

plt.tick_params(axis='both', labelsize=20)
plt.grid(True)
plt.show()
plt.savefig("error-radial.png")

In [ ]:
errorAverage_tangential = sum(pmt_shift_distance_array_tangential) / len(pmt_shift_distance_array_tangential)
fig = plt.figure(figsize=(12,12))
n, bins, patches = plt.hist(pmt_shift_distance_array_tangential, nbins, facecolor='g')
plt.xlabel('Tangential Error (cm)', size=25)
plt.ylabel('Frequency', size=25)

STD_tangential = np.std(pmt_shift_distance_array_tangential)
STD_tangential_error = STD_tangential / np.sqrt(len(pmt_shift_distance_array_tangential))
printSTD = "SD = " + str(round(STD_tangential,1)) + " cm"
printAverage = "AVG = " + str(round(errorAverage_tangential,2)) + " ± " + str(round(STD_tangential_error,2)) + " cm"

plt.title(printAverage+", "+printSTD, size=30, wrap=True)

plt.tick_params(axis='both', labelsize=20)
plt.grid(True)
plt.show()
plt.savefig("error-tangential.png")